In [1]:
%load_ext watermark
import pandas as pd
import xarray as xr
import langholzli as lhz
from myst_nb import glue

my_tag = "hammerdirt-analyst"

target_species = pd.read_csv('data/endpipe/target_species.csv')
# window extents
wext_one = (7.1673981838973591,47.0951027481864699)
wext_two = (7.3867370409999991,47.1906072286096219)

# grid extents
# square id
# 585220 Langholzli and vorholzli
# 585215 langholzli sud
# 590220 vorholzli nord

# extent of the two grid squares that make langholz
grid_one = (7.2407620202501777,47.0858405714567212)
grid_two = (7.3069424859821366,47.1758797299210784)
date_limit = '2000-01-01'

section = "D"
rule = " "

# citation of from GBIF download
# GBIF.org (29 January 2024) GBIF Occurrence Download  https://doi.org/10.15468/dl.fnsqgf


obs_cols = ['kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 
        'species', 'scientificName', 
        'verbatimScientificName',
        'publishingOrgKey', 'decimalLatitude',
        'decimalLongitude',
        'eventDate']

In [2]:
info_species_test = pd.read_csv("data/endpipe/Atlas5x5_585_215.csv", encoding='utf-16', sep=';')
info_species_test["Années (min)"] = info_species_test["Années (min)"].astype('str')
info_species_test["Années (max)"] = info_species_test["Années (max)"].astype('str')
caption = "Exemple de données pour la grille 585215 d'Infospecies: Infoflora."
sample_info = info_species_test.head().style.set_table_styles(lhz.table_css_styles).hide()

tablen = 1
table_one = lhz.add_table_to_page(sample_info, tablen, caption, section, format_index='columns')
glue('table_one', table_one, display=False)

In [3]:
summary_gbif = pd.read_csv("data/endpipe/initial_gbif_download_summary.csv", sep="\t")
summary_columns = ['dataset_title',  'number_records']
summary_gbif = summary_gbif[summary_columns].set_index('dataset_title').copy()
summary_gbif.rename(columns={"number_records" :"Nombre de données"}, inplace=True)
summary_gbif.index.name = None
tn = summary_gbif["Nombre de données"].sum()



summary_gbif = summary_gbif.style.set_table_styles(lhz.table_css_styles)


caption="Origine des observations d'espèces répertoriées sur le GBIF, nombre d'observations dans les grilles 585215 et 585220 = 24'567."
tablen = 2
table_two = lhz.add_table_to_page(summary_gbif, tablen, caption, section)
glue("table_two", table_two, display=False)

In [4]:
obs_lang = pd.read_csv("data/endpipe/gbif_observations_in_langholz_grids.csv")
display_cols = ['kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 
        'species', 'latitude',
        'longitude',
        'date']
obs_lang["eventDate"] = pd.to_datetime(obs_lang.eventDate)
obs_lang['date'] = obs_lang.eventDate.dt.date
rename_cols = {'decimalLatitude':'latitude', 'decimalLongitude':'longitude'}
obs_lang.rename(columns=rename_cols, inplace=True)

sample_gbif = obs_lang[display_cols].head().style.set_table_styles(lhz.table_css_styles).hide()
caption = "Exemple de données du GBIF utilisant les vertices des grilles 585215 et 585220 pour définir la surface."
tablen = 3
table_three = lhz.add_table_to_page(sample_gbif, tablen, caption, section, format_index='columns')
glue("table_three", table_three, display=False)

In [5]:
obs_l = pd.read_csv("data/endpipe/observations_langholzli_gbif.csv")

In [6]:
target_s = target_species["Scientific Name"].str.split().values
target_genus = [x[0] for x in target_s]
observed_genus = obs_lang[obs_lang.genus.isin(target_genus)].species.unique()
number_genus = len(observed_genus)

In [7]:
ds = xr.open_dataset("data/inprocess/main_data.nc")
data = ds.sel(logger=[214,208,209,228]).to_dataframe().reset_index()

gps = data[['logger', "X", 'Y']].drop_duplicates()
gps.to_csv("data/endpipe/temp_points.csv", index=False)

# Données

Le _Plan d’action pour la conservation d’espèces cibles dans les forêts claires_ est basé sur des recherches menées par des experts dans le domaine de la sylviculture et de la biodiversité. La mise en œuvre de ce plan s'appuie sur trois sources principales de données (page 8 Plan d'action):

1. Liste et descriptions des associations forestières selon NaiS (Frehner et al. 2005), [OFEV biodiversité](https://www.bafu.admin.ch/bafu/fr/home/themes/biodiversite/publications/publications-biodiversite/liste-especes-prioritaires-nationales.html)
2. Liste des milieux naturels prioritaires au niveau national (OFEV 2019), [OFEV biodiversité](https://www.bafu.admin.ch/bafu/fr/home/themes/biodiversite/publications/publications-biodiversite/liste-especes-prioritaires-nationales.html) 
3. Bases de données d’InfoSpecies, [demande de données](https://www.infospecies.ch/fr/donnees/)

Ces données sont accessibles au public. Dans les cas 1 et 2, les données sont la synthèse des observations et des connaissances spécifiques au domaine par des experts mandatés par la confédération et peuvent être téléchargées directement. Les données sur les espèces, quant à elles, sont une base de données d'observations du public gérée par Infospecies. Infospecies est l'association faîtière des Centres de données et d'informations et des Centres de coordination pour la conservation des espèces.

__Cartes__

Les cartes sont les couches composites de swissTLM3D, version 2.1. Les fichiers .shp utilisés sont disponibles dans le dossier cartographie du dépôt de l'application. Les cartes originales peuvent être téléchargées de l'Office fédéral de topographie à l'adresse suivante [swissTLM3D](https://www.swisstopo.admin.ch/fr/modele-du-territoire-swisstlm3d)

## Infospecies

Résolution des données Infospecies: les donées gratuites sont dans une résolution de 5x5 km.  Chaque carré de 5 km a un numéro d'identification à six chiffres qui peut être utilisé pour demander des données à la base de données infospecies.

Les identifiants de grille pour Längholzwald :

1. 585220 Langholzli and vorholzli
2. 585215 langholzli sud

Pour l'identification des espèces dans le périmètre défini, la résolution doit être plus grande. Cependant, nous pouvons nous faire une idée des espèces identifiées dans la région en considérant toutes les observations qui se trouvent à l'intérieur des deux carrés de la grille.



::::{grid} 1 1 1 1

:::{grid-item}
{glue}`table_one`
:::

:::{grid-item-card} 
:img-background: resources/maps/initial_map.jpeg
:::
:::{grid-item}
__Carte 1 :__ Examen initial des données accessibles au public. Les données pour le Längholzwald sont réparties entre deux carrés de grille (585220 et 585215). Notez que les données sur les espèces peuvent provenir de n'importe quel point de la grille.
:::
::::

## Global Biodiversity Information Facility (GBIF)

Selon Infospecies, les données sont aussi disponible sur [GBIF](https://www.gbif.org/). La source de ces données sur GBIF est le _Swiss National Biodiversity Data and Information Center_. Il s'agit d'une combinaison de données provenant de dix banques de données nationales différentes.

::::{grid} 1 1 2 2 
:::{grid-item}
:columns: 12 12 8 8

{glue}`table_two`
:::
:::{grid-item-card}
:columns: 12 12 4 4 

Global Biodiversity Information Facility (Système mondial d’information sur la biodiversité)—est un réseau international et une infrastructure de données financés par les gouvernements mondiaux ayant pour but de fournir à tous et partout un accès libre aux données sur toutes les formes de vie sur Terre.

Citation: GBIF.org (29 January 2024) GBIF Occurrence Download  https://doi.org/10.15468/dl.fnsqgf

:::
::::

::::{grid} 1 1 1 1

:::{grid-item}

{glue}`table_three`
:::

:::{grid-item-card} 
:img-background: resources/maps/initial_gbif_map.jpeg
:::
:::{grid-item}
__Carte 2 :__ Examen initial des données accessibles au public. Les données du GBIF indiquent les coordonnées GPS des observations. Cela nous permet de définir les observations qui ont eu lieu dans le Längholzwald. Nombre d'observations = 96.
:::

::::

## L'îlot de chaleur urbain

Le phénomène par lequel les zones urbaines enregistrent des températures nettement plus élevées que leurs environs ruraux. Cette augmentation de la température est due à des facteurs tels que la forte concentration de bâtiments, de routes et d'autres infrastructures qui absorbent et retiennent la chaleur, la végétation limitée, et les activités humaines générant de la chaleur. 



::::{grid} 1 1 2 2 
:::{grid-item-card}
:columns: 12 12 8 8
:img-background: resources/maps/initial_temp_map.jpeg



:::
:::{grid-item}
:columns: 12 12 4 4 

L'effet est plus prononcé pendant la nuit et peut entraîner une augmentation de la consommation d'énergie, des coûts de climatisation et des risques pour la santé en raison des vagues de chaleur.

Gabrial Erismann réalise son thèse de master sur la chaleur urbain. Voici quelques-uns des points de contrôle issus des données.

__Carte 3 :__ Examen initial des données accessibles au public : Température.
:::
::::

## Dechets

Les résultats de la dernière enquête fédérale sur les déchets indiquent un taux de 374 à 716 déchets par 100 mètres (p/100 m) pour Biel au bord du lac. Dans les Alpes bernoises, Grindelwald, ce chiffre est observé à environ 172 à 374 p/100 m. [IQAASL](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/) 
::::{grid} 1 1 1 1  
:::{grid-item-card} 
:img-background: resources/maps/biel_dechets.jpeg
:::

:::{grid-item}
__Carte 4 :__ Examen initial des données: Densité de déchets pour 100 m. Ce nombre a été mesuré autour des lacs et des rivières pendant 7 ans, les déchets le long des sentiers de randonnée ont été mesurés en unités par 100 m de sentier en 2021. Les valeurs du bassin de l'Aare sont en général plus faibles que celles d'autres lieux mesurés.

:::
::::

In [8]:
%watermark -d -u -gu "hammerdirt-analyst"

Github username: hammerdirt-analyst

Last updated: 2024-02-05

